<a href="https://colab.research.google.com/github/seongseopkim/anthouse/blob/main/LoRA_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리 설치

In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes


BART, 토크나이저 로딩 ( 원래 사용하던 모델 로딩 )

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_id = "facebook/bart-large-cnn"

# 토크나이저와 모델 로딩
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

print("✅ 모델과 토크나이저 로딩 완료!")


LoRA 설정

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

# LoRA 설정 정의
lora_config = LoraConfig(
    r=8,                        # 랭크: 작을수록 가벼움, 클수록 정밀
    lora_alpha=16,             # scaling factor
    lora_dropout=0.1,          # dropout 비율
    bias="none",               # bias는 LoRA 적용 안 함
    task_type=TaskType.SEQ_2_SEQ_LM,  # 시퀀스-투-시퀀스 작업 (요약)
    target_modules=["q_proj", "v_proj"]  # BART에서 LoRA를 적용할 레이어
)

# 기존 모델에 LoRA 구성 적용
model = get_peft_model(model, lora_config)

# 학습되는 파라미터 확인
model.print_trainable_parameters()


학습 파라미터 설정

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./lora-bart-checkpoints",   # 결과 저장 경로
    per_device_train_batch_size=2,          # 배치 크기 (적은 데이터면 2~4)
    per_device_eval_batch_size=2,
    learning_rate=1e-4,                     # 학습률
    num_train_epochs=3,                     # 학습 epoch 수
    logging_dir="./logs",                   # 로그 저장 경로
    save_strategy="steps",                  # 몇 step마다 저장할지
    evaluation_strategy="steps",            # 평가 주기
    eval_steps=100,                         # 평가 간격
    save_steps=200,                         # 저장 간격
    predict_with_generate=True,             # 요약 결과 생성도 평가에 포함
    fp16=True,                              # GPU가 지원하면 float16 사용
    report_to="none"                        # WandB 등 외부 로깅 끔
)

print("✅ 학습 파라미터 설정 완료!")


In [ ]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

# 데이터가 들어올 때 padding 등을 자동으로 맞춰주는 콜레이터
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 빈 Trainer 준비 (데이터셋은 나중에 연결할 거야)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=None,   # 나중에 데이터셋 연결
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("✅ Trainer 객체 생성 완료 (데이터셋은 나중에 연결)")


데이터셋 집어넣기

In [ ]:
from google.colab import files
import json

# Colab에서 파일 업로드
uploaded = files.upload()  # 여기서 lora_dataset.jsonl 선택

# JSONL 읽기
dataset = []
with open("lora_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        dataset.append(json.loads(line))

# 확인
print(dataset[0])
